In [1]:
import os
import shutil
from pathlib import Path
import random
import yaml

In [2]:
! pip install -U ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 2.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.4/214.4 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 38.8 MB/s eta 0:00:0000:01
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.6
    Uninstalling widgetsnbextension-3.6.6:
      Successfully uninstalled widgetsnbextension-3.6.6
  Attempting uninstall: jupyterlab-widgets
    Found existing installation: jupyterlab-widgets 3.0.9
    Uninstalling jupyterlab-widgets-3.0.9:
      Successfully uninstalled jupyterlab-widgets-3.0.9
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7.7.1


In [3]:
dataset_path = Path('/kaggle/input/atomichack')
output_path = Path('/kaggle/working/yolo_dataset/')
images_path = output_path / 'images'
labels_path = output_path / 'labels'

In [4]:
(images_path / 'train').mkdir(parents=True, exist_ok=True)
(images_path / 'val').mkdir(parents=True, exist_ok=True)
(labels_path / 'train').mkdir(parents=True, exist_ok=True)
(labels_path / 'val').mkdir(parents=True, exist_ok=True)

In [5]:
image_files = sorted([f for f in dataset_path.glob('*.jpg')])
annotation_files = sorted([f for f in dataset_path.glob('*.txt')])

In [6]:
data = list(zip(image_files, annotation_files))
random.shuffle(data)
image_files, annotation_files = zip(*data)

In [7]:
split_index = int(0.8 * len(image_files))
train_images = image_files[:split_index]
train_annotations = annotation_files[:split_index]
val_images = image_files[split_index:]
val_annotations = annotation_files[split_index:]

In [8]:
for img_file, ann_file in zip(train_images, train_annotations):
    shutil.copy(img_file, images_path / 'train' / img_file.name)
    shutil.copy(ann_file, labels_path / 'train' / ann_file.name)

for img_file, ann_file in zip(val_images, val_annotations):
    shutil.copy(img_file, images_path / 'val' / img_file.name)
    shutil.copy(ann_file, labels_path / 'val' / ann_file.name)

In [9]:
data_yaml = {
    'train': str(images_path / 'train'),
    'val': str(images_path / 'val'),
    'nc': 5,
    'names': ['adj', 'int', 'geo', 'pro', 'non']
}

In [10]:
with open(output_path / 'data.yaml', 'w') as yaml_file:
    yaml.dump(data_yaml, yaml_file, default_flow_style=False)

In [11]:
! pip install git+https://github.com/ultralytics/ultralytics.git@main

  Cloning https://github.com/ultralytics/ultralytics.git (to revision main) to /tmp/pip-req-build-rtz_jxk5
  Running command git clone --filter=blob:none --quiet https://github.com/ultralytics/ultralytics.git /tmp/pip-req-build-rtz_jxk5
  Resolved https://github.com/ultralytics/ultralytics.git to commit e5477713ddc5d5eaafb9985c2f5e5c7635119447
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for ultralytics: filename=ultralytics-8.2.32-py3-none-any.whl size=780638 sha256=db87cbf1b9e7ba2d8577ac2e8bac95912ba8c5ce7245a19c16383aaaf665ca5d
  Stored in directory: /tmp/pip-ephem-wheel-cache-3bhr47ui/wheels/60/db/e5/6abbdd13b4e3d5e2fdcd87690cd8738f842eeb37142fdccb5b
Successfully built ultralytics


In [12]:
import torch 
import torchvision

In [13]:
from ultralytics import YOLO

In [ ]:
model = YOLO('yolov8x.pt')

model.train(data = '/kaggle/working/yolo_dataset/data.yaml',
            epochs=20,
            imgsz=640,
            seed=42,
            batch=8,
            workers=4)

100%|██████████| 131M/131M [00:01<00:00, 87.4MB/s] 


Ultralytics YOLOv8.2.32 🚀 Python-3.10.13 torch-2.1.2+cpu CPU (Intel Xeon 2.20GHz)
engine/trainer: task=detect, mode=train, model=yolov8x.pt, data=/kaggle/working/yolo_dataset/data.yaml, epochs=20, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=4, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=42, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True

100%|██████████| 755k/755k [00:00<00:00, 4.27MB/s]


Overriding model.yaml nc=80 with nc=5

                   from  n    params  module                                       arguments                     
  0                  -1  1      2320  ultralytics.nn.modules.conv.Conv             [3, 80, 3, 2]                 
  1                  -1  1    115520  ultralytics.nn.modules.conv.Conv             [80, 160, 3, 2]               
  2                  -1  3    436800  ultralytics.nn.modules.block.C2f             [160, 160, 3, True]           
  3                  -1  1    461440  ultralytics.nn.modules.conv.Conv             [160, 320, 3, 2]              
  4                  -1  6   3281920  ultralytics.nn.modules.block.C2f             [320, 320, 6, True]           
  5                  -1  1   1844480  ultralytics.nn.modules.conv.Conv             [320, 640, 3, 2]              
  6                  -1  6  13117440  ultralytics.nn.modules.block.C2f             [640, 640, 6, True]           
  7                  -1  1   3687680  ultralytics

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

...

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


val: Scanning /kaggle/working/yolo_dataset/labels/val... 233 images, 26 backgrounds, 0 corrupt: 100%|██████████| 233/233 [00:00<00:00, 1089.16it/s]

val: New cache created: /kaggle/working/yolo_dataset/labels/val.cache


Plotting labels to runs/detect/train/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001111, momentum=0.9) with parameter groups 97 weight(decay=0.0), 104 weight(decay=0.0005), 103 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs/detect/train
Starting training for 20 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/20         0G      2.584      5.626      2.234         25        640:   1%|          | 1/117 [01:10<2:17:00, 70.87s/it]